### Load the model and make prediction
##### Input Args:

doc_dir = directory where the file with comments to be predicted

doc2vec_model_name = doc2vec model already trained

trained_clf = Trained classifier


In [9]:
from gensim.models import Doc2Vec
import re
import os
import random
from sklearn import metrics
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
def extract_words(sent):
    #print("line tokenizing : \n{0}".format(sent))
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) #strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remoce apostrophes
    sent = re.sub(r'\W',' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated space
    sent = sent.strip()
    return sent.split()

In [12]:
proj_root = os.path.dirname(os.getcwd())
#print("Current working dir : {0}".format(proj_root))

In [13]:
model_dir = proj_root + "/" + "model"
doc2vec_model = model_dir + "/" + 'comments.d2v'

#Load doc to vec model
doc2vec_model = Doc2Vec.load(doc2vec_model)

In [14]:
#Test the trained doc2vec model
cosine_similarity(
[doc2vec_model.infer_vector(extract_words("This is really exciting video. Thank you for presenting to us."))],
[doc2vec_model.infer_vector(extract_words("Exciting video. Keep it coming"))])

array([[0.7391128]], dtype=float32)

In [15]:
#Converts the comments to vector using doc2vec model trianed earlier
def get_doc2vec(model, doc_dir,comment_col_name='Comment'):
    comments_dfs = []
    
    for fname in sorted(os.listdir(doc_dir)):
        fname_full = doc_dir + "/" + fname
        print("file being read : {0}".format(fname_full))
        data = pd.read_csv(fname_full, sep='\t', header='infer')
        data['FNAME'] = fname
        data['WORDS'] = data[comment_col_name].map(lambda x: extract_words(x))
        data['SENTVECS'] = data['WORDS'].map(lambda x: model.infer_vector(x, steps=10))
        comments_dfs.append(data)

    comments = pd.concat(comments_dfs)
    return comments

In [16]:
doc_dir = "../indata"
comments = get_doc2vec(doc2vec_model, doc_dir)

file being read : ../indata/EdSheeran.csv
file being read : ../indata/Maroon5.csv
file being read : ../indata/SuperBowlLI.csv
file being read : ../indata/bing-bang-theory.csv
file being read : ../indata/bitcoin-vs-ethereum.csv
file being read : ../indata/coldpay.csv
file being read : ../indata/eminem-not-affraid.csv
file being read : ../indata/federer-rafael.csv
file being read : ../indata/formula1-2018.csv
file being read : ../indata/got8.csv
file being read : ../indata/house-of-cards.csv
file being read : ../indata/iphone-xs-vs-note9.csv
file being read : ../indata/mac-vs-windows.csv
file being read : ../indata/nfl-highlights.csv
file being read : ../indata/oculus.csv
file being read : ../indata/samsung-vr.csv
file being read : ../indata/tonight-show.csv
file being read : ../indata/tswift.csv
file being read : ../indata/warriors.csv


In [17]:
comments.head(5)

,Comment,CommentID,CreateTimeStamp,Type,videoID,videoTitle,FNAME,WORDS,SENTVECS
0,Like maroon Comment ed sheeran,UgxD46CBoJVhQCAibLp4AaABAg,2018-12-08T02:32:48.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[like, maroon, comment, ed, sheeran]","[0.10038329, -0.0014675149, 0.03223283, -0.225..."
1,Its Legendary December,Ugx91N6QKBVsR24qOH94AaABAg,2018-12-08T02:31:02.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[its, legendary, december]","[-0.008494277, -0.008857365, -0.06353382, -0.0..."
2,Feelings so deep in feelings,UgzG83e-1TszyKCMO8F4AaABAg,2018-12-08T02:28:16.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[feelings, so, deep, in, feelings]","[0.23715904, 0.11488962, 0.0027810452, 0.22044..."
3,I love your songs seriously I would like to me...,UgydHSuRFNFux7vGSQ94AaABAg,2018-12-08T02:20:23.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[i, love, your, songs, seriously, i, would, li...","[0.79949605, 0.7708359, 0.24607134, -0.2334915..."
4,I love it,UgwTYwXgN9KGAB7WIaN4AaABAg,2018-12-08T02:13:49.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[i, love, it]","[0.057383023, 0.14928776, 0.007573689, 0.01369..."


In [18]:
print("No of files : {0}".format(comments.FNAME.unique()))
print("No of Type : {}".format(comments.Type.unique()))

No of files : ['EdSheeran.csv' 'Maroon5.csv' 'SuperBowlLI.csv' 'bing-bang-theory.csv'
 'bitcoin-vs-ethereum.csv' 'coldpay.csv' 'eminem-not-affraid.csv'
 'federer-rafael.csv' 'formula1-2018.csv' 'got8.csv' 'house-of-cards.csv'
 'iphone-xs-vs-note9.csv' 'mac-vs-windows.csv' 'nfl-highlights.csv'
 'oculus.csv' 'samsung-vr.csv' 'tonight-show.csv' 'tswift.csv'
 'warriors.csv']
No of Type : ['Comment' 'Reply']


In [19]:
print("Number of qnique videos : {0}".format(len(comments.videoID.unique())))

Number of qnique videos : 916


In [21]:
trained_clf = model_dir + "/" + 'doc2vec_senti_rf.model'
loaded_clf = joblib.load(trained_clf)

In [22]:
comments['SENTIMENT_RF'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [23]:
comments.head(5)

,Comment,CommentID,CreateTimeStamp,Type,videoID,videoTitle,FNAME,WORDS,SENTVECS,SENTIMENT_RF
0,Like maroon Comment ed sheeran,UgxD46CBoJVhQCAibLp4AaABAg,2018-12-08T02:32:48.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[like, maroon, comment, ed, sheeran]","[0.10038329, -0.0014675149, 0.03223283, -0.225...",0
1,Its Legendary December,Ugx91N6QKBVsR24qOH94AaABAg,2018-12-08T02:31:02.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[its, legendary, december]","[-0.008494277, -0.008857365, -0.06353382, -0.0...",1
2,Feelings so deep in feelings,UgzG83e-1TszyKCMO8F4AaABAg,2018-12-08T02:28:16.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[feelings, so, deep, in, feelings]","[0.23715904, 0.11488962, 0.0027810452, 0.22044...",1
3,I love your songs seriously I would like to me...,UgydHSuRFNFux7vGSQ94AaABAg,2018-12-08T02:20:23.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[i, love, your, songs, seriously, i, would, li...","[0.79949605, 0.7708359, 0.24607134, -0.2334915...",1
4,I love it,UgwTYwXgN9KGAB7WIaN4AaABAg,2018-12-08T02:13:49.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[i, love, it]","[0.057383023, 0.14928776, 0.007573689, 0.01369...",1


In [24]:
trained_clf = model_dir + "/" + 'doc2vec_senti_kn.model'
loaded_clf = joblib.load(trained_clf)

In [25]:
comments['SENTIMENT_KN'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [26]:
comments.head(5)

,Comment,CommentID,CreateTimeStamp,Type,videoID,videoTitle,FNAME,WORDS,SENTVECS,SENTIMENT_RF,SENTIMENT_KN
0,Like maroon Comment ed sheeran,UgxD46CBoJVhQCAibLp4AaABAg,2018-12-08T02:32:48.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[like, maroon, comment, ed, sheeran]","[0.10038329, -0.0014675149, 0.03223283, -0.225...",0,0
1,Its Legendary December,Ugx91N6QKBVsR24qOH94AaABAg,2018-12-08T02:31:02.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[its, legendary, december]","[-0.008494277, -0.008857365, -0.06353382, -0.0...",1,1
2,Feelings so deep in feelings,UgzG83e-1TszyKCMO8F4AaABAg,2018-12-08T02:28:16.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[feelings, so, deep, in, feelings]","[0.23715904, 0.11488962, 0.0027810452, 0.22044...",1,1
3,I love your songs seriously I would like to me...,UgydHSuRFNFux7vGSQ94AaABAg,2018-12-08T02:20:23.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[i, love, your, songs, seriously, i, would, li...","[0.79949605, 0.7708359, 0.24607134, -0.2334915...",1,1
4,I love it,UgwTYwXgN9KGAB7WIaN4AaABAg,2018-12-08T02:13:49.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[i, love, it]","[0.057383023, 0.14928776, 0.007573689, 0.01369...",1,1


In [27]:
out_df = comments[['Comment','CommentID', 'CreateTimeStamp','Type', 'videoID', 'videoTitle', 'FNAME','SENTIMENT_RF','SENTIMENT_KN']]

In [28]:
out_df.head(5)

,Comment,CommentID,CreateTimeStamp,Type,videoID,videoTitle,FNAME,SENTIMENT_RF,SENTIMENT_KN
0,Like maroon Comment ed sheeran,UgxD46CBoJVhQCAibLp4AaABAg,2018-12-08T02:32:48.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,0,0
1,Its Legendary December,Ugx91N6QKBVsR24qOH94AaABAg,2018-12-08T02:31:02.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,1,1
2,Feelings so deep in feelings,UgzG83e-1TszyKCMO8F4AaABAg,2018-12-08T02:28:16.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,1,1
3,I love your songs seriously I would like to me...,UgydHSuRFNFux7vGSQ94AaABAg,2018-12-08T02:20:23.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,1,1
4,I love it,UgwTYwXgN9KGAB7WIaN4AaABAg,2018-12-08T02:13:49.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,1,1


In [29]:
out_file_name = '../outdata/video_comments_analysis_sentiments.csv'
out_df.to_csv(out_file_name, sep='\t', index=False)

In [41]:
print('****End of script****')

****End of script****
